# Simple Lyrics Downloader for Digital Humanities

This notebook shows students how to:
- Connect to the Genius API
- Download lyrics for individual songs
- Download lyrics for entire albums
- Save lyrics as text files for research

**What you need:**
- Free Genius API token from https://genius.com/api-clients

In [ ]:
# Install the library
%pip install lyricsgenius

In [ ]:
# Setup
import lyricsgenius
import time
import os
import re
from datetime import datetime

# Put your token here (you need to sign up for a Genius API account
# and create an API client to get your token)
GENIUS_TOKEN = 'YOUR_TOKEN_HERE'

# Connect to API with rate limiting (required for educational use)
genius = lyricsgenius.Genius(GENIUS_TOKEN)
genius.timeout = 20          # Wait up to 20 seconds for responses
genius.sleep_time = 2        # Wait 2 seconds between requests
genius.retries = 2           # Retry failed requests twice

print("✓ Connected to Genius API")

In [ ]:
# Download a single song
def download_song(artist_name, song_title):
    """Download lyrics for one song"""
    print(f"Downloading: {song_title} by {artist_name}")
    time.sleep(2)  # Rate limiting
    
    song = genius.search_song(song_title, artist_name)
    
    if song and song.lyrics:
        return {
            'title': song.title,
            'artist': song.artist,
            'lyrics': song.lyrics,
            'album': getattr(song, 'album', 'Unknown')
        }
    else:
        print(f"No lyrics found for {song_title}")
        return None

# Example - download one song
song_data = download_song("Taylor Swift", "Anti-Hero")
if song_data:
    print(f"✓ Downloaded: {song_data['title']}")
    print(f"Lyrics length: {len(song_data['lyrics'])} characters")

In [ ]:
# Download an entire album
def download_album(artist_name, album_name, max_songs=15):
    """Download all songs from an album"""
    print(f"Downloading album: {album_name} by {artist_name}")
    
    # Get artist with their songs
    artist = genius.search_artist(artist_name, max_songs=max_songs)
    
    if not artist:
        print(f"Artist {artist_name} not found")
        return []
    
    # Find songs from the specific album
    album_songs = []
    for song in artist.songs:
        song_album = getattr(song, 'album', '')
        if album_name.lower() in song_album.lower():
            song_data = {
                'title': song.title,
                'artist': song.artist,
                'lyrics': song.lyrics,
                'album': song_album
            }
            album_songs.append(song_data)
            print(f"✓ Found: {song.title}")
            time.sleep(3)  # Rate limiting between songs
    
    print(f"\n✓ Downloaded {len(album_songs)} songs from {album_name}")
    return album_songs

# Example - download an album (uncomment to use)
# album_songs = download_album("Taylor Swift", "Midnights", max_songs=10)
# print(f"Got {len(album_songs)} songs")

In [ ]:
# Save lyrics to text files
def save_lyrics(songs, folder_name="lyrics"):
    """Save songs as text files"""
    if not songs:
        print("No songs to save")
        return
    
    # Make sure songs is a list
    if isinstance(songs, dict):
        songs = [songs]
    
    # Create folder
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
        print(f"Created folder: {folder_name}")
    
    # Save each song
    for song in songs:
        if not song or not song.get('lyrics'):
            continue
            
        # Clean filename
        safe_title = re.sub(r'[^\w\s-]', '', song['title'])
        safe_artist = re.sub(r'[^\w\s-]', '', song['artist'])
        filename = f"{safe_artist} - {safe_title}.txt"
        filepath = os.path.join(folder_name, filename)
        
        # Save file
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(f"Title: {song['title']}\n")
            f.write(f"Artist: {song['artist']}\n")
            f.write(f"Album: {song['album']}\n")
            f.write(f"Downloaded: {datetime.now().strftime('%Y-%m-%d')}\n")
            f.write("=" * 40 + "\n\n")
            f.write(song['lyrics'])
        
        print(f"✓ Saved: {filename}")
    
    print(f"\n✓ All files saved in '{folder_name}' folder")

# Save the downloaded song
if 'song_data' in locals() and song_data:
    save_lyrics(song_data)
    print("Ready for analysis!")

## How to Use This Notebook

1. **Get a free API token** from https://genius.com/api-clients
2. **Replace** `'YOUR_TOKEN_HERE'` with your actual token
3. **Run the cells** in order

### Examples:

**Download one song:**
```python
song = download_song("Artist Name", "Song Title")
save_lyrics(song)
```

**Download an album:**
```python
album = download_album("Artist Name", "Album Name", max_songs=10)
save_lyrics(album, folder_name="my_research")
```

### Important Notes:
- Rate limiting (delays) are **required** - don't remove them
- Files are saved with metadata for academic citation
- Use downloaded lyrics for text analysis and distant reading research